In [1]:

import pandas as pd
import numpy as np
import os
import sys
from trade import self_uqer

In [2]:
self_uqer.name2code_index['HS300']

'000300.XSHG'

In [3]:
self_uqer.get_data_benchmark(self_uqer.name2code_index['HS300'], 'close', '20220201', '20220301')

2022-02-07    4634.0899
2022-02-08    4608.7729
2022-02-09    4652.0609
2022-02-10    4639.8625
2022-02-11    4601.3953
2022-02-14    4551.6878
2022-02-15    4600.1025
2022-02-16    4617.9893
2022-02-17    4629.1649
2022-02-18    4651.2358
2022-02-21    4634.3141
2022-02-22    4574.1527
2022-02-23    4623.0465
2022-02-24    4529.3189
2022-02-25    4573.4247
2022-02-28    4581.6459
2022-03-01    4619.6862
Name: close, dtype: float64

In [7]:
rawpath = '/Users/nymath/.rqalpha/bundle'

In [8]:
test =  pd.to_datetime(np.array(np.load(f'{rawpath}/trading_dates.npy'),dtype='U8'))

In [9]:
pd.Series(0,test)['2012'].index

DatetimeIndex(['2012-01-04', '2012-01-05', '2012-01-06', '2012-01-09',
               '2012-01-10', '2012-01-11', '2012-01-12', '2012-01-13',
               '2012-01-16', '2012-01-17',
               ...
               '2012-12-18', '2012-12-19', '2012-12-20', '2012-12-21',
               '2012-12-24', '2012-12-25', '2012-12-26', '2012-12-27',
               '2012-12-28', '2012-12-31'],
              dtype='datetime64[ns]', length=243, freq=None)

## 交易日数据获取 

In [48]:
from datetime import datetime, timedelta

In [170]:
from itertools import tee
from itertools import compress
def pairwise(iterable):
    # pairwise('ABCDEFG') --> AB BC CD DE EF FG
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

In [136]:
ddt[0]

Timestamp('2022-01-04 00:00:00')

In [135]:
type(ddt[0])

pandas._libs.tslibs.timestamps.Timestamp

In [52]:
ddt[0] + timedelta(days=11)

Timestamp('2022-01-15 00:00:00')

In [10]:
def get_TradeDate(start_date: str, 
                  end_date: str) -> pd.core.indexes.datetimes.DatetimeIndex:
    try:
        test = pd.to_datetime(np.array(np.load(f'{rawpath}/trading_dates.npy'),dtype='U8'))
    except:
        '数据源路径不存在, 请输入rqalpha download-bundle安装'
    trade_dates = pd.Series(0,index=test)[start_date:end_date].index
    return trade_dates

In [11]:
ddt = get_TradeDate('20220101', '20220302')

In [71]:
list(map(lambda x:x.year,ddt));
reduce(lambda x,y:x+y,[1,2,3])

6

In [58]:
ddt == datetime(2022,1,4)

array([ True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False])

In [59]:
ddt[0] + timedelta(days=-1)

Timestamp('2022-01-03 00:00:00')

In [64]:
np.where(ddt==datetime(2022,1,4))[0]

array([0])

In [111]:
current = '20220104'
period = 1

In [114]:
current = pd.to_datetime(current)
shifts = 3*period + int(np.sign(period)*10)
next = current + timedelta(days=shifts)

In [120]:
days = get_TradeDate(min(current,next),max(current, next))
days[2]

Timestamp('2022-01-06 00:00:00')

In [124]:
np.where(days == current)[0] + period

array([1])

In [130]:
current =  pd.to_datetime(current)
shifts = 3*period + int(np.sign(period)*10)
next = current + timedelta(days=shifts)
days = get_TradeDate(min(current,next),max(current, next))
index = np.where(days == current)[0] + period
result = days[int(index)]

In [178]:
def get_NextTradeDate(current,period=-1):
    current =  pd.to_datetime(current)
    shifts = 3*period + int(np.sign(period)*10)
    next_time = current + timedelta(days=shifts)
    days = get_TradeDate(min(current,next_time),max(current, next_time))
    index = np.where(days == current)[0] + period
    result = days[int(index)]
    return result

In [231]:
def get_TransferDate(start_date: str, 
                  end_date: str, end=True, freq='month') -> pd.core.indexes.datetimes.DatetimeIndex:
    def pairwise(iterable):
        # pairwise('ABCDEFG') --> AB BC CD DE EF FG
        a, b = tee(iterable)
        next(b, None)
        return zip(a, b)
    s = get_TradeDate(start_date, end_date)[0]
    e = get_TradeDate(start_date, end_date)[-1]
    if end:
        date_s = pd.to_datetime(s)
        date_e = pd.to_datetime(get_NextTradeDate(current=e,period=1))
        trade_date = get_TradeDate(date_s,date_e)
        exec(f'lst = list(map(lambda x:x[0]!=x[1],pairwise(map(lambda x:x.{freq},trade_date))))',locals())
        res = pd.to_datetime(list(compress(trade_date, locals()['lst'])))
        return res
    if not end:
        date_s = pd.to_datetime(get_NextTradeDate(current=s,period=-1))
        date_e = pd.to_datetime(e)
        trade_date = get_TradeDate(date_s,date_e)
        exec(f'lst = list(map(lambda x:x[0]!=x[1],pairwise(map(lambda x:x.{freq},trade_date))))',locals())
        res = pd.to_datetime(list(compress(trade_date, locals()['lst'])))
        return res

In [232]:
get_TransferDate('20220101','20220301')

DatetimeIndex(['2022-01-28', '2022-02-28'], dtype='datetime64[ns]', freq=None)

In [ ]:
# INterest

In [39]:
df = h5py.File('/Users/nymath/.rqalpha/bundle/yield_curve.h5',mode='r')
ddt = pd.DataFrame(df['data'][:])

In [60]:
def get_data_InterestRate(stard_date: str,
                          end_date: str,period: str ='10Y') -> pd.Series:
    with h5py.File(f'{rawpath}/yield_curve.h5',mode='r') as hf:
        df = pd.DataFrame(hf['data'][:])
    data = df[period]
    data.index = pd.to_datetime(np.array(df['date'],dtype='<U8'))
    df = data[stard_date:end_date]
    return df
    

In [44]:
ddt 

,date,0S,1M,2M,3M,6M,9M,1Y,2Y,3Y,...,6Y,7Y,8Y,9Y,10Y,15Y,20Y,30Y,40Y,50Y
0,20050104,0.027393,0.027712,0.028029,0.028344,0.029278,0.030195,0.031095,0.034527,0.037687,...,0.045542,0.047618,0.049423,0.050958,0.052221,0.054470,0.049944,0.020563,NaN,NaN
1,20050105,0.025691,0.025962,0.026232,0.026501,0.027300,0.028089,0.028868,0.031881,0.034730,...,0.042293,0.044487,0.046516,0.048382,0.050084,0.056134,0.058088,0.049705,NaN,NaN
2,20050106,0.024908,0.025210,0.025510,0.025809,0.026699,0.027577,0.028443,0.031788,0.034943,...,0.043268,0.045663,0.047867,0.049882,0.051707,0.057979,0.059500,0.048289,NaN,NaN
3,20050107,0.023603,0.023918,0.024231,0.024542,0.025469,0.026384,0.027286,0.030772,0.034060,...,0.042735,0.045232,0.047530,0.049630,0.051533,0.058077,0.059674,0.048026,NaN,NaN
4,20050110,0.024315,0.024628,0.024939,0.025249,0.026170,0.027079,0.027976,0.031438,0.034702,...,0.043302,0.045771,0.048042,0.050114,0.051988,0.058377,0.059803,0.047760,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4476,20221125,0.009901,0.012452,0.019614,0.020002,0.020422,0.020477,0.020685,0.022871,0.024154,...,NaN,0.028215,NaN,NaN,0.028300,0.029677,0.030459,0.032162,0.032633,0.03295
4477,20221128,0.010021,0.012455,0.019611,0.020002,0.020422,0.020484,0.020699,0.023114,0.024304,...,NaN,0.028516,NaN,NaN,0.028600,0.029977,0.030697,0.032337,0.032796,0.03310
4478,20221129,0.012198,0.013265,0.020056,0.020221,0.020641,0.020813,0.021139,0.023519,0.024715,...,NaN,0.028737,NaN,NaN,0.028800,0.030177,0.030960,0.032663,0.033134,0.03345
4479,20221130,0.014500,0.015567,0.019987,0.020811,0.020941,0.021016,0.021341,0.023602,0.024717,...,NaN,0.028745,NaN,NaN,0.028850,0.030227,0.031035,0.032763,0.033234,0.03355


# Benckmark

In [250]:
def get_data_benchmark(ts_code: str, field: str,
                       start_date: str, end_date: str):
    '''
    tscode: 传入标准的指数代码, 例如 000300.XSHG
    field: open, high, low, close, volumn, total_turnover
    '''
    with h5py.File(f'{rawpath}/indexes.h5',mode='r') as hf:
        df = pd.DataFrame(hf[ts_code][:])
    data = df[field]
    data.index = pd.to_datetime(np.array(df['datetime'],dtype='<U8'))
    df = data.loc[start_date:end_date]
    return df

In [270]:
get_data_benchmark('000300.XSHG', 'close', '20220101', '20220110')

2022-01-04    4917.7653
2022-01-05    4868.1202
2022-01-06    4818.2318
2022-01-07    4822.3689
2022-01-10    4844.0451
Name: close, dtype: float64

In [237]:
df = h5py.File('/Users/nymath/.rqalpha/bundle/indexes.h5',mode='r')

In [245]:
pd.DataFrame(df['000300.XSHG'][:])

,datetime,open,close,high,low,volume,total_turnover
0,20050104000000,994.7700,982.7900,994.7700,980.6600,7.412869e+08,4.431977e+09
1,20050105000000,981.5800,992.5600,997.3200,979.8800,7.119109e+08,4.529208e+09
2,20050106000000,993.3300,983.1700,993.7900,980.3300,6.288028e+08,3.921015e+09
3,20050107000000,983.0500,983.9600,995.7100,979.8100,7.298694e+08,4.737469e+09
4,20050110000000,983.7600,993.8800,993.9600,979.7900,5.791697e+08,3.762932e+09
...,...,...,...,...,...,...,...
4348,20221125000000,3751.2822,3775.7764,3790.5660,3743.4138,1.429297e+10,1.917110e+11
4349,20221128000000,3701.9020,3733.2424,3736.0813,3672.1306,1.278257e+10,1.963056e+11
4350,20221129000000,3766.5702,3848.4224,3854.7742,3766.5702,1.787014e+10,2.890623e+11
4351,20221130000000,3836.5598,3853.0365,3867.6591,3833.8272,1.575114e+10,2.500515e+11


In [253]:
import rqalpha

In [257]:
rqalpha.api.get_price('000300.XSHG', '20220101', end_date='20220301')

RuntimeError: stack is empty

In [263]:
df = pd.DataFrame(np.ones(20).reshape(10,2),columns=['a','b'])

In [15]:
cond = """
c = 1+1
d = 1+2
print(c,d)
"""

In [11]:
exec(cond)

2 3


In [16]:
def main():
    exec(cond)

In [22]:
tset = """
for i in range(10):
    print(i) # test
"""

In [27]:
test = pd.Series(index=[0,1,2,3,4,5],dtype='f8')

In [28]:
test[0]

nan

In [ ]:
def get_data_benchmark(ts_code: str, field: str,
                       start_date: str, end_date: str):
    '''
    tscode: 传入标准的指数代码, 例如 000300.XSHG
    field: open, high, low, close, volumn, total_turnover
    '''
    with h5py.File(f'{rawpath}/indexes.h5',mode='r') as hf:
        df = pd.DataFrame(hf[ts_code][:])
    data = df[field]
    data.index = pd.to_datetime(np.array(df['datetime'],dtype='U8'))
    df = data.loc[start_date:end_date]
    return df

In [1]:
import h5py
rawpath = '/Users/nymath/.rqalpha/bundle'
import pandas as pd
import numpy as np
from trade import self_uqer


# 获取股票数据

In [45]:
ts_code = '000001.XSHE'
current_date = '20100101'

In [49]:
with h5py.File(f'{rawpath}/ex_cum_factor.h5',mode='r') as hf:
    df = pd.DataFrame(hf[ts_code][:])
df.drop(0,axis='rows',inplace=True)
df.index = pd.to_datetime(np.array(df['start_date'],dtype='U8'))
df = df['ex_cum_factor']
true_time = df.index.asof(current_date) 
ex_factor = df[-1]/df[true_time]

In [36]:
df.index = pd.to_datetime(np.array(df['start_date'],dtype='U8'))

In [38]:
df.index = pd.to_datetime(np.array(df['start_date'],dtype='U8'))

In [64]:
ts = pd.date_range('20020101',periods=10)

In [4]:
from trade import self_uqer

In [4]:
lst = self_uqer.get_data_index_components('000300.XSHG','20220101')

In [8]:
lsss = []
for s in lst[:30]:   
    lsss.append(self_uqer.get_data_assets(s, 'close', '20120101', '20200101',exclude_ds=True))

In [2]:
self_uqer.get_data_assets('000001.XSHE', 'close', '20120101', '20120301',exclude_ds=True)

2012-01-04    4.781361
2012-01-05    4.853902
2012-01-06    4.841286
2012-01-09    4.976905
2012-01-10    4.976905
2012-01-11    4.976905
2012-01-12    4.976905
2012-01-13    4.976905
2012-01-16    4.976905
2012-01-17    4.976905
2012-01-18    4.976905
2012-01-19    4.976905
2012-01-20    5.292298
2012-01-30    5.270221
2012-01-31    5.248143
2012-02-01    5.188218
2012-02-02    5.336453
2012-02-03    5.358531
2012-02-06    5.308068
2012-02-07    5.232373
2012-02-08    5.330145
2012-02-09    5.330145
2012-02-10    5.333299
2012-02-13    5.260759
2012-02-14    5.235527
2012-02-15    5.241835
2012-02-16    5.188218
2012-02-17    5.188218
2012-02-20    5.188218
2012-02-21    5.200834
2012-02-22    5.241835
2012-02-23    5.244989
2012-02-24    5.320684
2012-02-27    5.336453
2012-02-28    5.440533
2012-02-29    5.393224
2012-03-01    5.434225
dtype: float64

In [9]:
lst = pd.Series([1,2,3,4],index='')

In [12]:
ts_code = '000001.XSHE'
with h5py.File(f'{rawpath}/ex_cum_factor.h5',mode='r') as hf:
    df = pd.DataFrame(hf[ts_code][:])

In [14]:
df.iloc[0,0] = '19900101000000'

In [15]:
df

,start_date,ex_cum_factor
0,19900101000000,1.00000
1,19910817000000,1.15749
2,19920323000000,1.74700
3,19930524000000,3.32715
4,19940711000000,5.32344
5,19950925000000,6.58838
6,19960527000000,13.17680
7,19970825000000,19.89110
8,19991018000000,20.43190
9,20001106000000,22.97620
